# Build a local Open Interpreter server for a custom front end


In [1]:
# Development Environment Verification
# This cell helps verify that we're using the local development version of the interpreter

import os
import sys

def verify_dev_environment():
    """
    Verify the development environment setup for open-interpreter
    Returns:
        bool: True if everything is correctly set up
    """
    # 1. Check Python Interpreter and Environment
    print("1. Python Environment:")
    print(f"   Python Executable: {sys.executable}")
    
    # 2. Check Working Directory
    current_dir = os.getcwd()
    project_root = os.path.dirname(current_dir)
    print("\n2. Directory Information:")
    print(f"   Current Directory: {current_dir}")
    print(f"   Project Root: {project_root}")
    
    # 3. Ensure Project Root is in Python Path
    if project_root not in sys.path:
        sys.path.insert(0, project_root)
        print("\n3. Added Project Root to Python Path")
    else:
        print("\n3. Project Root already in Python Path")
    
    # 4. Import and Verify Interpreter
    try:
        import interpreter
        interpreter_path = interpreter.__file__
        print("\n4. Interpreter Import:")
        print(f"   Path: {interpreter_path}")
        
        # Verify if using local development version
        is_dev_version = project_root in interpreter_path
        print(f"   Using Development Version: {is_dev_version}")
        
        if not is_dev_version:
            print("   WARNING: Not using the local development version!")
            return False
            
    except ImportError:
        print("\n4. ERROR: Failed to import interpreter!")
        return False
    
    # 5. Display first few Python path entries
    print("\n5. Python Path (first 5 entries):")
    for path in sys.path[:5]:
        print(f"   - {path}")
    
    return True

# Run verification
is_dev_env_ready = verify_dev_environment()
print("\nEnvironment Verification:", "✅ PASSED" if is_dev_env_ready else "❌ FAILED")

1. Python Environment:
   Python Executable: /Users/clay/Library/Caches/pypoetry/virtualenvs/open-interpreter-2ItoNx9d-py3.9/bin/python

2. Directory Information:
   Current Directory: /Users/clay/Code/open-interpreter/examples
   Project Root: /Users/clay/Code/open-interpreter

3. Project Root already in Python Path

4. Interpreter Import:
   Path: /Users/clay/Code/open-interpreter/interpreter/__init__.py
   Using Development Version: True

5. Python Path (first 5 entries):
   - /Users/clay/.pyenv/versions/3.9.5/lib/python39.zip
   - /Users/clay/.pyenv/versions/3.9.5/lib/python3.9
   - /Users/clay/.pyenv/versions/3.9.5/lib/python3.9/lib-dynload
   - 
   - /Users/clay/Library/Caches/pypoetry/virtualenvs/open-interpreter-2ItoNx9d-py3.9/lib/python3.9/site-packages

Environment Verification: ✅ PASSED


In [ ]:
# Install required packages if not already installed
%pip install flask python-dotenv requests boto3

In [ ]:
# Flask Server Setup for Open Interpreter
# This server provides a REST API interface for the Open Interpreter

from flask import Flask, request, jsonify
from interpreter import interpreter
from dotenv import load_dotenv
import json
import os
import logging
from threading import Thread

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

app = Flask(__name__)

# Load environment variables
load_dotenv()

# Check required environment variables
required_env_vars = ['LITELLM_MODEL']  # Add other required env vars as needed
missing_vars = [var for var in required_env_vars if not os.getenv(var)]

if missing_vars:
    logger.error(f"Missing required environment variables: {', '.join(missing_vars)}")
    logger.info("Please ensure the following variables are set in your .env file:")
    for var in missing_vars:
        logger.info(f"  {var}=your-key-here")

# Interpreter Configuration
interpreter.llm.model = os.environ.get('LITELLM_MODEL', 'gpt-3.5-turbo')
interpreter.llm.context_window = 10000
interpreter.llm.max_tokens = 4096
interpreter.auto_run = True
interpreter.verbose = True

# Chat endpoint
@app.route('/chat', methods=['POST'])
def chat():
    """
    Handle chat requests
    Expected payload: {"prompt": "User's message or question"}
    Returns: {
        "response": "AI's response",
        "code": "Generated code (if any)",
        "output": "Code execution output (if any)"
    }
    """
    data = request.json
    prompt = data.get('prompt')
    
    logger.info(f"Request received - IP: {request.remote_addr}, Prompt: {prompt}")
    
    if not prompt:
        logger.warning("Empty prompt received")
        return jsonify({"error": "No prompt provided"}), 400

    if missing_vars:
        error_msg = f"Server configuration error: Missing required variables {', '.join(missing_vars)}"
        logger.error(error_msg)
        return jsonify({"error": error_msg}), 500

    full_response = ""
    current_code_block = []
    code_blocks = []
    execution_outputs = []
    
    try:
        for chunk in interpreter.chat(prompt, stream=True, display=False):
            if isinstance(chunk, dict):
                chunk_type = chunk.get("type")
                content = chunk.get("content", "")
                
                if chunk_type == "message":
                    # 如果之前有未完成的代码块，先保存它
                    if current_code_block:
                        code_blocks.append("".join(current_code_block))
                        current_code_block = []
                    full_response += content
                elif chunk_type == "code":
                    # 收集代码块片段
                    current_code_block.append(content)
                elif chunk_type == "output":
                    execution_outputs.append(content)
            
            elif isinstance(chunk, str):
                try:
                    json_chunk = json.loads(chunk)
                    full_response += json_chunk.get("response", "")
                except json.JSONDecodeError:
                    full_response += chunk
        
        # 确保最后的代码块也被保存
        if current_code_block:
            code_blocks.append("".join(current_code_block))
        
        # 清理代码块，移除多余的空行和格式问题
        cleaned_code_blocks = []
        for code in code_blocks:
            # 移除多余的空行
            lines = [line for line in code.split('\n') if line.strip()]
            # 重新组合代码，确保格式正确
            cleaned_code = '\n'.join(lines)
            cleaned_code_blocks.append(cleaned_code)
        
        logger.info(f"Request processed successfully - Prompt: {prompt[:50]}...")
        return jsonify({
            "response": full_response.strip(),
            "code": "\n\n".join(cleaned_code_blocks) if cleaned_code_blocks else None,
            "output": "\n".join(execution_outputs) if execution_outputs else None
        })
    
    except Exception as e:
        error_msg = str(e)
        logger.error(f"Error processing request - Prompt: {prompt}, Error: {error_msg}")
        return jsonify({"error": f"Error processing request: {error_msg}"}), 500
# Shutdown endpoint
@app.route('/shutdown', methods=['GET'])
def shutdown():
    """Shutdown the Flask server gracefully"""
    func = request.environ.get('werkzeug.server.shutdown')
    if func is None:
        raise RuntimeError('Not running with the Werkzeug Server')
    func()
    return 'Server shutting down...'

# Server control functions
server_thread = None

def start_server():
    """Start the Flask server in a background thread"""
    global server_thread
    if server_thread and server_thread.is_alive():
        print("Server is already running")
        return
    
    def run_flask_app():
        app.run(host='0.0.0.0', port=6000)
    
    server_thread = Thread(target=run_flask_app)
    server_thread.daemon = True
    server_thread.start()
    logger.info(f"Current model: {interpreter.llm.model}")
    print("Open Interpreter server is running on http://0.0.0.0:6000")

def stop_server():
    """Stop the Flask server gracefully"""
    import requests
    try:
        requests.get('http://localhost:6000/shutdown')
        print("Server stopped successfully")
    except:
        print("Server already stopped or not running")

# Start the server in background thread
start_server()

2025-02-15 00:09:32 [INFO] Current model: bedrock/anthropic.claude-3-sonnet-20240229-v1:0


 * Serving Flask app '__main__'
Open Interpreter server is running on http://0.0.0.0:6000
 * Debug mode: off


2025-02-15 00:09:32 [INFO] WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:6000
 * Running on http://192.168.86.246:6000
2025-02-15 00:09:32 [INFO] Press CTRL+C to quit
2025-02-15 00:10:25 [INFO] 127.0.0.1 - - [15/Feb/2025 00:10:25] "POST /v1/chat/completions HTTP/1.1" 404 -
2025-02-15 00:10:41 [INFO] 127.0.0.1 - - [15/Feb/2025 00:10:41] "POST /v1/chat/completions HTTP/1.1" 404 -
2025-02-15 00:12:16 [INFO] Request received - IP: 127.0.0.1, Prompt: 你好，请帮我检查当前设备的系统信息。
00:12:16 - LiteLLM:INFO: utils.py:2749 - 
LiteLLM completion() model= anthropic.claude-3-sonnet-20240229-v1:0; provider = bedrock
2025-02-15 00:12:16 [INFO] 
LiteLLM completion() model= anthropic.claude-3-sonnet-20240229-v1:0; provider = bedrock
2025-02-15 00:12:17 [INFO] Found credentials in environment variables.
2025-02-15 00:12:17 [INFO] HTTP Request: POST https://bedrock-runtime.us

In [4]:
def test_interpreter(prompt):
    """
    Test the Open Interpreter server with formatted output
    Args:
        prompt: The prompt to send to the server
    """
    try:
        response = requests.post(
            'http://localhost:6000/chat',
            json={"prompt": prompt},
            headers={"Content-Type": "application/json"}
        )
        
        if response.status_code == 200:
            result = response.json()
            
            # Display the response in a structured way using Markdown
            md_output = "## AI Response\n"
            md_output += result.get("response", "No response") + "\n"
            
            if result.get("code"):
                md_output += "\n## Generated Code\n```python\n"
                # 确保代码块格式正确
                code = result.get("code").strip()
                md_output += code + "\n```\n"
            
            if result.get("output"):
                md_output += "\n## Execution Output\n```\n"
                # 清理输出格式
                output = result.get("output").strip()
                md_output += output + "\n```\n"
            
            display(Markdown(md_output))
        else:
            print(f"Error: {response.status_code}")
            print(response.json().get('error', 'Unknown error occurred'))
            
    except requests.exceptions.ConnectionError:
        print("Error: Could not connect to server. Make sure the server is running on port 5001.")
    except Exception as e:
        print(f"Error: {str(e)}")

In [ ]:
# Server control with proper shutdown mechanism
import requests
import signal
import os

def test_shutdown():
    """
    Test the server shutdown endpoint with process termination
    Returns:
        bool: True if shutdown was successful
    """
    try:
        # First try to find the process using port 5001
        import psutil
        for proc in psutil.process_iter(['pid', 'name', 'connections']):
            try:
                for conn in proc.connections():
                    if conn.laddr.port == 5001:
                        print(f"Found server process: {proc.pid}")
                        # Send SIGTERM signal to the process
                        os.kill(proc.pid, signal.SIGTERM)
                        print("Server shutdown signal sent successfully")
                        return True
            except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
                continue
                
        print("No server process found on port 5001")
        return False
        
    except Exception as e:
        print(f"Error during shutdown: {str(e)}")
        return False

# Execute shutdown test
test_shutdown()

: 